In [29]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

import pandas as pd
import numpy as np 
from tabulate import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import metrics
nltk.download('vader_lexicon')
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from sklearn.decomposition import LatentDirichletAllocation


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/prashantkarna/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [30]:
# Load the dataset
data = pd.read_csv("/Users/prashantkarna/Documents/Research Materials/Datasets/Suicide_Detection.csv")


In [31]:

data = data.drop('Unnamed: 0',axis=1)

In [32]:
data['class'] = data['class'].replace(['suicide'], 1)
data['class'] = data['class'].replace(['non-suicide'], 0)

In [33]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prashantkarna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
X = data.drop('class', axis=1)
y = data['class']


In [35]:
# To remove emails
email_regex = r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
regexes_to_remove = [email_regex, r'Subject:', r'Re:']

for i in range(0, len(X)):
    # removing all special charachter
    review = re.sub('[^a-zA-Z]', ' ', str(X['text'][i]))
    # make document as lowerCase
    review = review.lower()
    # splitting the documents into words for ex ['iam', 'omar']
    review = review.split()
    # make lemmatization --> (change, changing, changes)---> (change)
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    # join the document agian
    review = ' '.join(review)
    
    # removing mails
    for r in regexes_to_remove:
        X['text'][i] = re.sub(r, '', review)

In [41]:
# Unigram parameter grid
param_grid_unigram = {"C": [0.01, 0.1, 1, 10, 100]}

# Bigram parameter grid
param_grid_bigram = {"C": [0.01, 0.1, 1, 10, 100]}

# Trigram parameter grid
param_grid_trigram = {"C": [0.01, 0.1, 1, 10, 100]}


In [42]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
tfidf_vectorizer_uni = TfidfVectorizer(max_features=10000,ngram_range=(1,1))
tfidf_vectorizer_bi = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
tfidf_vectorizer_tri = TfidfVectorizer(max_features=10000, ngram_range=(1,3))

X_tfidf = tfidf_vectorizer.fit_transform(X['text'])
# X_tfidf_test = tfidf_vectorizer.transform(X['text'])

X_tfidf_uni = tfidf_vectorizer_uni.fit_transform(X['text'])
# X_tfidf_test_uni = tfidf_vectorizer_uni.transform(X['text'])

X_tfidf_bi = tfidf_vectorizer_bi.fit_transform(X['text'])
# X_tfidf_test_bi =tfidf_vectorizer_bi.transform(X'text'])

X_tfidf_tri = tfidf_vectorizer_tri.fit_transform(X['text'])
# X_tfidf_test_tri = tfidf_vectorizer_tri.transform(X_test['text'])

In [43]:
# Unigram parameter tuning
lr_unigram = LogisticRegression(max_iter=10000, solver='saga')
grid_unigram = GridSearchCV(lr_unigram, param_grid_unigram, cv=5)
grid_unigram.fit(X_tfidf_uni, y)
print("Best hyperparameters for unigram model:", grid_unigram.best_params_)

# Bigram parameter tuning
lr_bigram = LogisticRegression(max_iter=10000, solver='saga')
grid_bigram = GridSearchCV(lr_bigram, param_grid_bigram, cv=5)
grid_bigram.fit(X_tfidf_bi, y)
print("Best hyperparameters for bigram model:", grid_bigram.best_params_)

# Trigram parameter tuning
lr_trigram = LogisticRegression(max_iter=10000, solver='saga')
grid_trigram = GridSearchCV(lr_trigram, param_grid_trigram, cv=5)
grid_trigram.fit(X_tfidf_tri , y)
print("Best hyperparameters for trigram model:", grid_trigram.best_params_)


Best hyperparameters for unigram model: {'C': 10}
Best hyperparameters for bigram model: {'C': 1}
Best hyperparameters for trigram model: {'C': 1}


In [44]:
# Evaluate the unigram model
unigram_score = grid_unigram.score(X_tfidf_uni, y)
print("Unigram score:", unigram_score)

# Evaluate the bigram model
bigram_score = grid_bigram.score(X_tfidf_bi, y)
print("Bigram score:", bigram_score)

# Evaluate the trigram model
trigram_score = grid_trigram.score(X_tfidf_tri, y)
print("Trigram score:", trigram_score)



Unigram score: 0.9498737471668519
Bigram score: 0.9441772882787387
Trigram score: 0.9439101321130329


In [45]:
from sklearn.metrics import classification_report

# Unigram evaluation
y_pred_unigram = grid_unigram.predict(X_tfidf_uni)
print("Unigram evaluation:")
print(classification_report(y, y_pred_unigram))

# Bigram evaluation
y_pred_bigram = grid_bigram.predict(X_tfidf_bi)
print("Bigram evaluation:")
print(classification_report(y, y_pred_bigram))

# Trigram evaluation
y_pred_trigram = grid_trigram.predict(X_tfidf_tri)
print("Trigram evaluation:")
print(classification_report(y, y_pred_trigram))


Unigram evaluation:
              precision    recall  f1-score   support

           0       0.94      0.96      0.95    116037
           1       0.95      0.94      0.95    116037

    accuracy                           0.95    232074
   macro avg       0.95      0.95      0.95    232074
weighted avg       0.95      0.95      0.95    232074

Bigram evaluation:
              precision    recall  f1-score   support

           0       0.94      0.95      0.94    116037
           1       0.95      0.94      0.94    116037

    accuracy                           0.94    232074
   macro avg       0.94      0.94      0.94    232074
weighted avg       0.94      0.94      0.94    232074

Trigram evaluation:
              precision    recall  f1-score   support

           0       0.94      0.95      0.94    116037
           1       0.95      0.94      0.94    116037

    accuracy                           0.94    232074
   macro avg       0.94      0.94      0.94    232074
weighted avg   